# Protein prepatation
by Mauro Álvarez, Albert Meseguer, Adrià Pérez and Cristina Prat

Currently with the function prepareProtein of htmd we use propka and pdb2pqr to guess the protonation states and hydrogens. The idea of this project is to write a different algorithm based more or less on this strategy (of course looking at how pdb2pqr could also be useful).

We need the atomic information of the different amino acids in order to know if they have to be protonated.

|Amino acids           |Hydrogen donor atoms|Hydrogen acceptor atoms|
|----------------------|--------------------|-----------------------|
|Arginine (Arg, R)     |NE, NH1 (2), NH2 (2)|                       |
|Asparagine (Asn, N)   |ND2 (2)             |OD1 (2)                |
|Aspartic acid (Asp, D)|                    |OD1 (2), OD2 (2)       |
|Glutamine (Gln, Q)    |NE2 (2)             |OE1 (2)                |
|Glutamic acid (Glu, E)|                    |OE1 (2), OE2 (2)       |
|Histidine (His, H)    |ND1, NE2            |ND1, NE2               |
|Lysine (Lys, K)       |NZ (3)              |                       |
|Serine (Ser, S)       |OG                  |OG (2)                 |
|Threonine (Thr, T)	   |OG1                 |OG1 (2)                |
|Tryptophan (Trp, W)   |NE1                 |                       |
|Tyrosine (Tyr, Y)     |OH                  |OH                     |


The molecule produced by the preparation step has residue names modified according to their protonation. Later system-building functions assume these residue names. Note that support for alternative charge states varies between the forcefields.

Charge +1    |  Neutral   | Charge -1
-------------|------------|----------
 -           |  ASH       | ASP
 -           |  CYS       | CYM
 -           |  GLH       | GLU
HIP          |  HID/HIE   |  -
LYS          |  LYN       |  -
 -           |  TYR       | TYM
ARG          |  AR0       |  -


First we import the modules we are going to need for the project

In [1]:
from htmd import *

Please cite -- HTMD: High-Throughput Molecular Dynamics for Molecular Discovery
J. Chem. Theory Comput., 2016, 12 (4), pp 1845-1852. 
http://pubs.acs.org/doi/abs/10.1021/acs.jctc.6b00049


You are on the latest HTMD version (1.3.1).


For this report is used the Trypsin molecule as an example

In [2]:
mol = Molecule('3PTB')

2016-06-29 13:28:02,362 - htmd.molecule.molecule - INFO - Attempting PDB query for 3PTB


We need to remove the water and others ligands that we would have in our molecule, for this reason, we end up with just the protein.

In [3]:
mol.filter("protein")

2016-06-29 13:28:06,929 - htmd.molecule.molecule - INFO - Removed 72 atoms. 1629 atoms remaining in the molecule.


After that, we make a copy of that molecule because we just want to take into account five amino acids: ARG, LYS, ASP, GLU, HIS, CYS and TYR. They are interesting because of their hidrogen donor/acceptor activity.

In [4]:
copied = mol.copy()
copied.filter('resname ARG LYS ASP GLU HIS CYS TYR')

2016-06-29 13:28:08,627 - htmd.molecule.molecule - INFO - Removed 1175 atoms. 454 atoms remaining in the molecule.


We create two new variables (*rid* and *rn*) that contain resids and resname in order to easily iterate afterwards.

In [5]:
rid = np.unique(copied.get('resid'))
rn = copied.get('resname', "name CA")

In [6]:
# Firstly, a pos_list (basic aa interactions) and neg_list (acid aa interactions) are created
pos_list = []
neg_list = []
for resname, resid in zip(rn, rid):
    # Let's start with GLU
    if resname == "GLU":
        # Select resids that can interact with basic or acid resids of the ARG LYS ASP GLU HIS CYS TYR amino acids 
        # by h-bonds
        a = mol.get("resid", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 4 of name OE1 OE2
                    and resid "+str(resid))
        # Remove redudant outcomes
        un = np.unique(a)
        for residueID in un:
            try:
                l = len(np.unique(mol.get("resname", sel="resid "+str(residueID))))
                # There are some positions in the sequence that have two residues instead of one. In order to avoid 
                # that, we calculate the distance, if it is bigger than 1 means that there are two residues at the 
                # same position. At this point, under the possible positions and if there are an acid, we get it and
                # we add it at neg_list
                if l > 1:
                    for i in range(0, l):
                        if np.unique(mol.get("resname", sel="resid "+str(residueID)))[i] in ["GLU", "ASP"] and residueID != resid:
                            res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                            # Both lists have the same format: resname and resid of our residue, and after that, resname 
                            # and resid of the residue that interacts our residue
                            tup = (resname, resid, res, residueID)
                            neg_list.append(tup)
                            print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                else:
                    # If there is no multiple amino acid positions, we add normally the residues in the list
                    if np.unique(mol.get("resname", sel="resid "+str(residueID))) in ["GLU", "ASP"] and residueID != resid:
                        res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                        tup = (resname, resid, res, residueID)
                        neg_list.append(tup)
                        print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
            except:
                # In the case that are some mistakes, it will show a print which indicate the position
                print("Error at " + np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                continue
                
    elif resname == "ASP":
        a = mol.get("resid", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 4 of name OD1 OD2 and resid "+str(resid))
        #print(np.unique(a), "resid", resid)
        un = np.unique(a)
        for residueID in un:
            try:
                l = len(np.unique(mol.get("resname", sel="resid "+str(residueID))))
                if l > 1:
                    for i in range(0, l):
                        if np.unique(mol.get("resname", sel="resid "+str(residueID)))[i] in ["GLU", "ASP"] and residueID != resid:
                            res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                            tup = (resname, resid, res, residueID)
                            neg_list.append(tup)
                            print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                else:
                    
                    if np.unique(mol.get("resname", sel="resid "+str(residueID))) in ["GLU", "ASP"] and residueID != resid:
                        res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                        tup = (resname, resid, res, residueID)
                        neg_list.append(tup)
                        print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
            except:
                print("Error at " + np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                continue
                
    elif resname == "ARG":
        a = mol.get("resid", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 4 of name NH1 NH2 NE and resid "+str(resid))
        #print(np.unique(a), "resid", resid)
        un = np.unique(a)
        for residueID in un:
            try:
                l = len(np.unique(mol.get("resname", sel="resid "+str(residueID))))
                if l > 1:
                    print("aaaaaaaaaaa")
                    for i in range(0, l):
                        if np.unique(mol.get("resname", sel="resid "+str(residueID)))[i] in ["LYS", "ARG"] and residueID != resid:
                            res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                            tup = (resname, resid, res, residueID)
                            pos_list.append(tup)
                            print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                else:
                    
                    if np.unique(mol.get("resname", sel="resid "+str(residueID))) in ["LYS", "ARG"] and residueID != resid:
                        res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                        tup = (resname, resid, res, residueID)
                        pos_list.append(tup)
                        print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
            except:
                print("Error at " + np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                continue
                
    elif resname == "LYS":
        a = mol.get("resid", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 4 of name NZ and resid "+str(resid))
        #print(np.unique(a), "resid", resid)
        un = np.unique(a)
        for residueID in un:
            try:
                l = len(np.unique(mol.get("resname", sel="resid "+str(residueID))))
                if l > 1:
                    for i in range(0, l):
                        if np.unique(mol.get("resname", sel="resid "+str(residueID)))[i] in ["LYS", "ARG"] and residueID != resid:
                            res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                            tup = (resname, resid, res, residueID)
                            pos_list.append(tup)
                            print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                else:
                    
                    if np.unique(mol.get("resname", sel="resid "+str(residueID))) in ["LYS", "ARG"] and residueID != resid:
                        res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                        tup = (resname, resid, res, residueID)
                        pos_list.append(tup)
                        print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
            except:
                print("Error at " + np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                continue

['ASP'] 71
['GLU'] 80
['GLU'] 70
['GLU'] 77


In [7]:
print(pos_list)

[]


In [8]:
print(neg_list)

[('GLU', 70, array(['ASP'], dtype=object), 71), ('GLU', 70, array(['GLU'], dtype=object), 80), ('GLU', 80, array(['GLU'], dtype=object), 70), ('GLU', 80, array(['GLU'], dtype=object), 77)]


In [9]:
def redundancy_eliminator(lista):
    # The aim is to remove the redudance in this list
    # We use three list: salidas have the positions of the residues that we are, llegadas have the positions 
    # of the residues which interactue and, eliminar have the index of the element in the list that are redundant
    salidas = []
    llegadas = []
    eliminar = []
    if len(lista) > 1: # If we have redudancies
        for i in range(0, len(lista)):
            if lista[i][1] in llegadas and lista[i][3] in salidas: #Entonces tendremos redundancia
                # Keep the index of the list elements which are redundants
                eliminar.append(i)
                next
            else:
                # Those elements that are not redundat, we append it in the list to allow their comparation
                # in future iterations
                salidas.append(lista[i][1])
                llegadas.append(lista[i][3])

    for element in eliminar:
        lista.remove(lista[element])

    return lista
            

In [10]:
neg_list = redundancy_eliminator(neg_list)
print(neg_list)

[('GLU', 70, array(['ASP'], dtype=object), 71), ('GLU', 70, array(['GLU'], dtype=object), 80), ('GLU', 80, array(['GLU'], dtype=object), 77)]


In [11]:
pos_list = redundancy_eliminator(pos_list)
print(pos_list)

[]
